In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import string

import os

In [ ]:
pip install nltk

In [32]:
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer


In [ ]:
os.getcwd()

In [3]:
path = os.path.join(os.path.dirname(os.getcwd()),'raw_data')

In [4]:
print(path)

/Users/camillemolen/code/mfaruki/replenish/raw_data


In [33]:
df = pd.read_json(os.path.join(path,'train.json'))

In [ ]:
def lemma(text):
    for t in text:
        text_lst = t.split()
        print(text_lst)
        noun_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "n") for tok in text_lst]
    return noun_lemmatized

In [ ]:
df

In [ ]:
def preprocessing(tokenized_sentence):
    stop_words = set(stopwords.words('english'))
    no_stopwords = [w for w in tokenized_sentence if not w in stop_words] 
    #new = ' '.join(no_stopwords)
    
    #token_sentence= word_tokenize(new)
    # Lemmatizing the verbs
    verb_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "v") for tok in no_stopwords]
    # 2 - Lemmatizing the nouns
    noun_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "n")for tok in verb_lemmatized]

    ingredients = []
    for word in noun_lemmatized:
        ingredients.append(word.replace(" ","_"))
    final_ingredients = ' '.join(ingredients)
    return final_ingredients



In [ ]:
#df

In [ ]:
#df['clean']=df['ingredients'].apply(preprocessing)

In [34]:
df

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...,...
39769,29109,irish,"[light brown sugar, granulated sugar, butter, ..."
39770,11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
39771,2238,irish,"[eggs, citrus fruit, raisins, sourdough starte..."
39772,41882,chinese,"[boneless chicken skinless thigh, minced garli..."


K Means Testing

In [35]:
# Step 1: Preprocess the ingredients strings
#df['clean']=df['ingredients'].map(lambda x: ' '.join(x))
#df['clean'] = df['ingredients'].apply(lambda x: x.split(', '))
ingredients_list = df['ingredients'].tolist()

In [36]:
df

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...,...
39769,29109,irish,"[light brown sugar, granulated sugar, butter, ..."
39770,11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
39771,2238,irish,"[eggs, citrus fruit, raisins, sourdough starte..."
39772,41882,chinese,"[boneless chicken skinless thigh, minced garli..."


In [37]:
# Step 2: Convert ingredients into a bag-of-words representation
vectorizer = TfidfVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=0.005)
X = vectorizer.fit_transform(ingredients_list)

/Users/camillemolen/.pyenv/versions/3.10.6/envs/replenish/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
#df_new = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out()).T
#df_new['sum'] = df_new.sum(axis=1)
#df_new[['sum']].sort_values('sum', ascending=False).head(50)

In [22]:
# Step 3: Scale the data
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X.toarray())
#X_scaled

In [38]:
# Step 4: Apply KMeans clustering
num_clusters = 10 # You can adjust this based on your desired number of clusters
kmeans = KMeans(n_clusters=num_clusters)

In [39]:
kmeans_res = kmeans.fit_predict(X)

/Users/camillemolen/.pyenv/versions/3.10.6/envs/replenish/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [40]:
set(kmeans_res)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [49]:
#column with clusters
df['cluster'] = kmeans.fit_predict(X)

In [50]:
df['cluster'].value_counts()

cluster
4    10672
2     5277
6     3990
9     3847
7     3351
8     3090
1     2553
0     2534
3     2268
5     2192
Name: count, dtype: int64

In [51]:
pd.set_option('display.max_colwidth', 200)

In [52]:
df[df['cluster']==9][['ingredients', 'cluster', 'cuisine']].head(30)

,ingredients,cluster,cuisine
11,"[low sodium soy sauce, fresh ginger, dry mustard, green beans, white pepper, sesame oil, scallions, canola oil, sugar, Shaoxing wine, garlic, ground turkey, water, crushed red pepper flakes, corn ...",9,chinese
19,"[soy sauce, vegetable oil, red bell pepper, chicken broth, yellow squash, garlic chili sauce, sliced green onions, broccolini, salt, fresh lime juice, cooked rice, chicken breasts, corn starch]",9,vietnamese
23,"[sesame seeds, red pepper, yellow peppers, water, extra firm tofu, broccoli, soy sauce, orange bell pepper, arrowroot powder, fresh ginger, sesame oil, red curry paste]",9,chinese
25,"[sugar, lo mein noodles, salt, chicken broth, light soy sauce, flank steak, beansprouts, dried black mushrooms, pepper, chives, oyster sauce, dark soy sauce, peanuts, sesame oil, cabbage]",9,chinese
28,"[green bell pepper, egg roll wrappers, sweet and sour sauce, corn starch, molasses, vegetable oil, oil, soy sauce, shredded cabbage, garlic, onions, fresh ginger root, ground pork, carrots]",9,chinese
37,"[chicken broth, cooking oil, chinese five-spice powder, ground black pepper, salt, sugar, crushed garlic, chicken thighs, soy sauce, star anise]",9,filipino
61,"[lemongrass, large garlic cloves, rice, unsweetened coconut milk, fresh ginger, peanut sauce, spareribs, sesame oil, tamari soy sauce, golden brown sugar, dry sherry, boiling water]",9,thai
66,"[jasmine rice, garlic, scallions, sugar, shiitake, Gochujang base, beansprouts, top round steak, sesame seeds, rice vinegar, carrots, soy sauce, sesame oil, Taiwanese bok choy]",9,korean
78,"[top round steak, vegetable oil, shiitake, soy sauce, fresh asparagus, green onions]",9,japanese
110,"[chinese rice wine, shiitake, sesame oil, oil, soy sauce, egg roll wrappers, garlic, corn starch, sugar, ground black pepper, ground pork, carrots, fresh ginger, cooking oil, salt, cabbage]",9,chinese


In [53]:
df.cluster.unique_values()

AttributeError: 'Series' object has no attribute 'unique_values'

In [54]:
kmeans_res.get_params

AttributeError: 'numpy.ndarray' object has no attribute 'get_params'